### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [1]:
from sqp import *

TESTSTRING = "02002102000101000000102002110000021102201100000121101100102200102212002011112010100202020220021011110001020002102020211001102210020111001100000102100022100110201210022100020000101002210202100021000220220211202211110002221011010211000211202201021002102200012201101110222110002022012210202020020102100202211110202001122020000110020222220022110010020002102120002010010000211002021102102121210202221122000110202101020002020022200021000211020211022210200121022200010211002201101110220220110202110202210020212102102120002210002202112110210020001010002002000202102121222022121022201210211202020022100222101102112100221202021001010211020210102110202211200202000000000022102020000021111220012110201121010002002020000120200222022110202011002101002110010002120221100011000002100220222202021110222102200022001101011122021021111120021100010210222100222110202210102002221000021202020210200201101001120002211121011000212002000122022200121011120000210111011111020112221002002202"
# TESTSTRING = TESTSTRING[:40]
numpy.printoptions(suppress=True)

The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [2]:
def run_sqp(test_string, lmbd):
    global pbm
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l), np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    x0 = pbm.TransitionMatrix.reshape(-1)
    # x0 = np.random.rand(n**2)
    x0 = np.hstack((x0, np.dot(C, x0))).copy()
    global res
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-12, 'disp': True, "maxiter":5000},
            bounds=bounds, 
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [20]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.03)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)
print("Eqn con rhs: ")
print(pbm.EqnCon(res.x))
print("Ineq con rhs: ")
print(pbm.IneqCon(res.x))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6018.290807882225
            Iterations: 46
            Function evaluations: 445
            Gradient evaluations: 46
The original MLE estimated transition matrix is: 


array([[0.40342298, 0.17359413, 0.42298289],
       [0.5019305 , 0.34362934, 0.15444015],
       [0.38566553, 0.33788396, 0.27645051]])

The New Transiiton Matrix after the SQP is: 


array([[4.85561763e-01, 1.24900090e-14, 5.14438237e-01],
       [2.42780881e-01, 2.69475258e-15, 7.57219119e-01],
       [5.14438237e-01, 2.42780881e-01, 2.42780881e-01]])

Eqn con rhs: 
[1.55431223e-14 2.22044605e-16 0.00000000e+00]
Ineq con rhs: 
[-1.68962067e-15  1.73258848e-03 -1.42247325e-16 -1.32359401e-15
  1.62994414e-02  1.73258848e-03 -9.43689571e-16 -4.43221848e-16
  3.08662942e-02  1.45668529e-02  1.74895271e-15  4.54331471e-02
  3.08662942e-02  1.45668529e-02  1.45668529e-02  3.63424568e-16
 -5.67254577e-16  1.45668529e-02  2.02699094e-15  3.44342610e-16
 -1.05124243e-15 -1.51614832e-15  3.08662942e-02  1.62994414e-02
  3.85983971e-15  3.63164360e-15  4.54331471e-02  3.08662942e-02
  1.45668529e-02  1.45668529e-02 -2.46157261e-15 -1.92380833e-15
 -1.26721550e-15  1.96891115e-16  5.55111512e-17  1.87886229e-15
  2.91337058e-02 -1.52308721e-15  1.45668529e-02  2.91337058e-02
 -1.45369827e-15 -1.44849410e-15  1.45668529e-02  1.45668529e-02
  3.81639165e-17  3.59868385e-15  2.33666810e-15  2.50147125e-15
 -4.83987850e-16  8.17054757e-16 -5.55978874e-16  1.62994414e-02
  3.08662942e-02  1.37390099e-15  2.80067761e-15  1.62994414e-02
  1.62994414e-

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [11]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.0334)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 7759.572681409751
            Iterations: 79
            Function evaluations: 716
            Gradient evaluations: 78
The original MLE estimated transition matrix is: 


array([[0.40342298, 0.17359413, 0.42298289],
       [0.5019305 , 0.34362934, 0.15444015],
       [0.38566553, 0.33788396, 0.27645051]])

The New Transiiton Matrix after the SQP is: 


array([[3.52010129e-01, 3.01550514e-01, 3.46439357e-01],
       [8.25728374e-15, 2.67286193e-14, 1.00000000e+00],
       [4.04020323e-01, 2.97989839e-01, 2.97989839e-01]])